In [ ]:
Repasaremos el ejemplo de la última diapositiva paso a paso.

1. Establecer una conexión a la base de datos

conn = psycopg2.connect(dbname='db', user='grok')

Este comando inicializa una nueva sesión de base de datos y devuelve un objeto de conexión. Tenemos que especificar el nombre de la base de datos y el nombre del usuario. Tenga en cuenta que dbnamees el nombre de la base de datos, no una tabla en la base de datos.

A lo largo de este módulo llamaremos a nuestra base de datos 'db'. En su máquina local, usaría su nombre de cuenta de usuario para el archivo user. Aquí vamos a utilizar 'grok'.

2. Crea un objeto cursor

cursor = conn.cursor()
El cursor es el objeto que interactúa con la base de datos. Podemos ejecutar consultas SQL y recibir su salida a través de este objeto. Podemos llamar a las funciones del objeto usando la .notación de punto ( ) tal como lo hacemos con los módulos. Las dos funciones que usaremos con mayor frecuencia son executey fetchall.

3. Ejecute una consulta SQL

cursor.execute('SELECT 2 + 3;')
Para ejecutar una consulta SQL, llamamos a la executefunción, que es una función del objeto cursor. Esta función toma la consulta SQL en forma de cadena como argumento.

4. Recibir la devolución de la consulta

records = cursor.fetchall()
La fetchallfunción devuelve el resultado de la última consulta. Al llevar datos de SQL a Python, los tipos de datos se convierten a la coincidencia más cercana en los tipos de datos de Python. Echaremos un vistazo más de cerca a esto más tarde.

Para comenzar con el uso básico de Psycopg2, escriba una función llamada select_allque consulte nuestra tabla Staro Planeten PostgreSQL y devuelva todas las filas usando la siguiente consulta:

 
SELECT * FROM Star;
Su función debe tomar el nombre de la tabla como un argumento de cadena, por lo que puede llamarla como para acceder a la Startabla:

>>> select_all('Star')
[(2713049, 5996, 0.956), (3114167, 5666, 0.677), (3115833, 5995, 0.847), ...]
O así para la Planetmesa:

>>> select_all('Planet')
[(10666592, 'K00002.01', 'Kepler-2b', 'CONFIRMED', 2.204735365, 16.39, 2025), ...]
Debería devolver el resultado de directamente. cursor.fetchall()

Como ha visto en el último problema, los datos de las consultas SQL en Psycopg2 se devuelven en forma de listas de Python. En el último problema, solicitó la tabla completa Stary Planet, que devolvió una lista de n tuplas de longitud m , donde m es el número de columnas y n es el número de filas en estas tablas.

Una lista de tuplas no se puede usar de la misma manera que, por ejemplo, una matriz Numpy 2D. Por ejemplo, el siguiente método de indexación para acceder al primer elemento no funcionará:

 
a = [(1, 2, 3), (4, 5, 6)]
print(a[0, 0])
En cambio, tenemos que usar el []operador dos veces: primero para acceder al primer elemento de la lista, es decir, la primera tupla, y luego para acceder al primer elemento de esa tupla:
 
a = [(1, 2, 3), (4, 5, 6)]
print(a[0][0])
Usando este método de indexación, podemos acceder a cada elemento de datos individual. Esto nos permite, por ejemplo, extraer columnas enteras de los datos recorriendo las filas. El siguiente fragmento de código muestra un ejemplo que extrae la t_effcolumna de la Startabla completa y la agrega a una nueva lista:

 
import psycopg2

conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Star')
records = cursor.fetchall()

t_eff = []
for row in records:
  t_eff.append(row[1])

print(t_eff)
import psycopg2
​
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Star')
records = cursor.fetchall()
​
t_eff = []
for row in records:
  t_eff.append(row[1])
​
print(t_eff)

Ahora que hemos visto cómo trabajar con los resultados de las consultas, podemos echar un vistazo más de cerca a los datos en sí. En la actividad anterior, aprendimos sobre diferentes tipos de datos en SQL cuando estábamos configurando tablas.

¿Cómo se convierten estos tipos de datos SQL en tipos de Python?

Echemos un vistazo a los Planettipos de datos de la tabla. Podemos usar una consulta que seleccione todas las columnas pero solo una fila:

 
SELECT * FROM Planet LIMIT 1;
En Python, esta consulta devolverá una lista que contiene una sola tupla. Podemos recorrer las entradas de esta tupla y llamar a la typefunción para determinar los tipos de datos:

 
import psycopg2
​
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
​
cursor.execute('SELECT * FROM Planet LIMIT 1;')
​
records = cursor.fetchall()
​
for col in records[0]:
    print(type(col))
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'float'>
<type 'float'>
<type 'int'>
La conversión de tipo de estos tipos es sencilla: SQL SMALLINTy INTEGERse convierten en enteros de Python, CHARcadenas VARCHARde Python y FLOATflotantes de Python.

Consulte la documentación de Psycopg2 cuando desee obtener más información sobre la conversión de tipos.

Una vez que tenemos los datos numéricos de la base de datos en Python, podemos escribirlos en matrices NumPy.

Dado que a menudo tratamos con datos de diferentes tipos en las bases de datos, es importante recordar que, si bien las listas y las tuplas de Python pueden contener datos de diferentes tipos, las matrices de NumPy no pueden.

Para convertir una lista de Python en una matriz NumPy simple, debemos asegurarnos de que la lista solo contenga datos de un tipo. Aparte de eso, los resultados de SQL se pueden cargar fácilmente en matrices NumPy:

 
import psycopg2
import numpy as np
​
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
​
cursor.execute('SELECT radius FROM Star;')
​
records = cursor.fetchall()
array = np.array(records)
​
print(array.shape)
print(array.mean())
print(array.std())
(66, 1)
0.886863636364
0.237456527847
Una vez que los datos se almacenan en matrices NumPy, tenemos acceso a todas las funciones de NumPy para manipular y analizar nuestros datos. Una cosa que ahora podemos hacer fácilmente es, por ejemplo, calcular una mediana.

Escriba una función llamada column_statsque calcule la media y la mediana de una columna seleccionada en una tabla Staro Planet. Para esto, deje que su función tome dos argumentos de cadena:

El nombre de las mesa;
el nombre de la columna.
y hacer que devuelva la media y la mediana (en este orden) de la columna seleccionada.

Cuando llama a su función, por ejemplo, en la t_effcolumna de la Startabla, la llamada y el retorno de la función deberían verse así:

>>> column_stats('Star', 't_eff')
(5490.681818181818, 5634.0)
Puede comparar su cálculo con la consulta SQL pura:

 
SELECT AVG(t_eff) FROM Star;
+-----------------------+
|          avg          |
+-----------------------+
| 5490.6818181818181818 |
+-----------------------+
(1 row)
​

Para establecer la conexión con la base de datos, usamos la misma rutina que antes. Para obtener la columna deseada de las tablas individuales, usamos la concatenación de cadenas para crear la consulta. Por último, convertimos los datos obtenidos en matrices NumPy para calcular convenientemente la media y la mediana de los datos.

Solución de muestra
import numpy as np
import psycopg2
​
def column_stats(table, col):
  conn = psycopg2.connect(dbname='db', user='grok')
  cursor = conn.cursor()
​
  query = 'SELECT ' + col + ' FROM ' + table + ';'
  cursor.execute(query)
  column = np.array(cursor.fetchall())
  return np.mean(column), np.median(column)

En este curso, ha aprendido dos enfoques diferentes para manejar datos. Lo que elija para un proyecto en particular depende de una variedad de factores, incluidas las preguntas que está planteando a los datos o si está utilizando una base de datos pública o un catálogo.

Hemos visto que SQL es conveniente de usar para muchas cosas, pero ¿cuán conveniente es exactamente? ¿Podemos hacer lo mismo en Python?

Repasemos algunos problemas en los que implementamos consultas SQL típicas de las actividades anteriores en Python. Comenzaremos con una consulta simple y agregaremos un nuevo elemento en cada problema.

Su primera tarea es replicar la siguiente consulta SQL:

 
SELECT kepler_id, radius
FROM Star
WHERE radius > 1.0;
Los datos se almacenan en , con el en la primera columna y el en la última. stars.csvkepler_idradius

Escriba una función llamada queryque tome el nombre del archivo CSV como argumento y devuelva los datos en una matriz NumPy de 2 columnas. Por ejemplo, este pequeño archivo CSV:

estrellas.csv
10666592,6350,1.991
10682541,5339,0.847
10797460,5850,1.04
su queryfunción debería funcionar de la siguiente manera:

>>> query('stars.csv')
array([[  1.06665920e+07   1.99100000e+00]
       [  1.07974600e+07   1.04000000e+00]])
Los datos numéricos se convierten automáticamente en flotantes en este procedimiento, no se preocupe si no se parece a la salida de SQL.

Pista
Puede usar la función loadtxt de NumPy con el usecolsargumento opcional para leer solo las columnas que le interesan.

Consultar bases de datos
Manejo de datos del cursor
Tipos de datos en SQL y Python
Escribir datos en matrices NumPy
Una mediana adecuada
(Aprobado)
SQL vs Python (o cualquier otra cosa)
SQL frente a Python
Consultas simples en Python 1/3
(Aprobado)
Consultas simples en Python 2/3
Consultas simples en Python 3/3
La herramienta adecuada para cada trabajo
¡Felicidades!
¡Felicidades!
Semana 5: Construcción de un clasificador de regresión
5a
Semana 5: Mejorando y evaluando nuestro clasificador
5b
Semana 6: Explorando la clasificación de aprendizaje automático
6a
Simple queries in Python 2/3
Instrucciones

Correr

Terminal

Reiniciar

Ahorrar

Mark
 
Agreguemos otro elemento a nuestra consulta. Ordene la tabla resultante en orden ascendente para que coincida con el resultado que obtendría con:

 
SELECT kepler_id, radius
FROM Star
WHERE radius > 1.0
ORDER BY radius ASC;
Puede usar sus resultados del último problema y luego construir sobre eso. Nuevamente, la función debe tener un nombre queryy debe tomar el nombre del archivo como argumento.


Pista
Puede usar la función argsort de NumPy para resolver este problema. Echa un vistazo a cómo funciona:

 
import numpy as np
  
a = np.array([3, 1, 2, 0])
b = np.argsort(a)
print(b)
print(a[b])
Devuelve los índices de la matriz no ordenada aen sus nuevas posiciones ordenadas. Puede pasar esta matriz devuelta ba la matriz original apara reorganizar sus entradas.

Agreguemos otro elemento más a nuestra consulta. Une la Startabla con la Planettabla y calcula la relación de tamaño, es decir, el radio del planeta/radio de la estrella para cada par de estrellas y planetas. Su queryfunción debería producir el mismo resultado que la consulta SQL:

 
SELECT p.radius/s.radius AS radius_ratio
FROM Planet AS p
INNER JOIN star AS s USING (kepler_id)
WHERE s.radius > 1.0
ORDER BY p.radius/s.radius ASC;
Puede usar sus resultados del último problema y luego construir sobre eso. La función debe tener un nombre query, pero esta vez debe tomar dos nombres de archivo como argumentos, para las estrellas y los planetas.

En planets.csv, la primera columna es el kepler_idy la penúltima columna es el radius.

Su función debe ser un vector de columna de proporciones, como este:

>>> query('stars.csv', 'planets.csv')
array([[  0.48798799],
       [  0.8260447 ],
       [  0.96209913],
       [  1.1556384 ],
       [  1.30403969],
       ...
Pista
Es posible que deba usar un bucle anidado para comparar cada planeta kepler_idcon cada estrella kepler_id. Una vez que haya encontrado una coincidencia y el radio de la estrella sea mayor que uno, puede agregar la proporción a la lista o matriz de resultados.

Este problema es mucho más difícil que los dos que teníamos antes, pero afortunadamente podemos reutilizar todo lo de la solución anterior. Comenzamos filtrando y ordenando los Stardatos. Luego, recorremos la nueva Starmatriz y encontramos los planetas correspondientes para cada estrella. Para esto, usamos wherela función de NumPy que devuelve una matriz de valores booleanos que satisfacen la condición que le damos como argumento. Una vez más, la matriz de retorno de esta función se puede utilizar con fines de indexación, produciendo solo aquellos planetas que coincidan con la iteración del bucle actual kepler_id. Agregamos todos los planetas coincidentes a una nueva matriz y devolvemos una versión ordenada de esta matriz.

Solución de muestra
import numpy as np

def query(fname_1, fname_2):
  stars = np.loadtxt(fname_1, delimiter=',', usecols=(0, 2))
  planets = np.loadtxt(fname_2, delimiter=',', usecols=(0, 5))

  f_stars = stars[stars[:,1]>1, :]                
  s_stars = f_stars[np.argsort(f_stars[:, 1]), :] 
 
  final = np.zeros((1, 1))
  for i in range(s_stars.shape[0]):
    kep_id = s_stars[i, 0]
    s_radius = s_stars[i, 1]

    matching_planets = planets[np.where(planets[:, 0] == kep_id), 1].T
    final = np.concatenate((final, matching_planets/s_radius))

  return np.sort(final[1:], axis = 0)
    
    

 
if __name__ == '__main__':
  print(query('stars.csv', 'planets.csv')) 
    
import numpy as np
​
def query(fname_1, fname_2):
  stars = np.loadtxt(fname_1, delimiter=',', usecols=(0, 2))
  planets = np.loadtxt(fname_2, delimiter=',', usecols=(0, 5))
​
  f_stars = stars[stars[:,1]>1, :]                
  s_stars = f_stars[np.argsort(f_stars[:, 1]), :] 
 
  final = np.zeros((1, 1))
  for i in range(s_stars.shape[0]):
    kep_id = s_stars[i, 0]
    s_radius = s_stars[i, 1]
​
    matching_planets = planets[np.where(planets[:, 0] == kep_id), 1].T
    final = np.concatenate((final, matching_planets/s_radius))
​
  return np.sort(final[1:], axis = 0)
    
    
​
 
if __name__ == '__main__':
  print(query('stars.csv', 'planets.csv')) 
    
​

trucciones

Correr

Terminal

Reiniciar

Ahorrar

Mark
Los últimos tres problemas mostraron que Python es fácil de usar para consultas simples, pero se vuelve mucho más difícil a medida que las consultas se vuelven más complejas. La última pregunta sobre las uniones fue especialmente difícil de implementar en Python, mientras que es relativamente simple en SQL.

Esto no debería sorprender, ya que eso es exactamente para lo que está diseñado SQL y lo que deberíamos usar para estos problemas. Sin embargo, existen buenas razones por las que es posible que no desee abandonar Python por completo para el trabajo relacionado con la base de datos.

Una cosa importante a considerar es que SQL está desarrollado para acceder a datos y las funciones integradas solo admiten operaciones matemáticas básicas. Más allá de eso se vuelve muy complicado.

Un buen ejemplo de esto es el cálculo de la mediana, que hemos hecho un par de veces en Python. Sin embargo, no hay funciones integradas para la mediana en SQL, y hacerlo a mano en SQL se vuelve bastante complicado. Ni siquiera hemos cubierto suficiente SQL para comprender cómo implementar una mediana, pero si está interesado, consulte este artículo de Postgresql que muestra ejemplos de cómo se podría implementar una mediana.